In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
import pymc3 as pm
import scipy as sp
import pyccl as ccl
import os
import utils
from make_data import MakeData
import getdist
from getdist import plots, MCSamples
from os import listdir
from os.path import isfile, join
import yaml
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['axes.labelsize'] = 14
plt.rc('legend',fontsize=14)

%matplotlib inline

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


# Load data

In [2]:
names_planck  = ['H0_gp', 'omega_m', 's80', 'S80']
labels_planck = [r'H_0', r'\omega_{\rm{m}}', r'\sigma_{\rm{8}}',  r'\S_{\rm{8}}']

planck1 = np.loadtxt('/home/jaimerz/Master/PLANCK/Planck_TTTEEE_lowl_lowE/base_plikHM_TTTEEE_lowl_lowE_post_zre6p5_1.txt')
planck2 = np.loadtxt('/home/jaimerz/Master/PLANCK/Planck_TTTEEE_lowl_lowE/base_plikHM_TTTEEE_lowl_lowE_post_zre6p5_2.txt')
planck3 = np.loadtxt('/home/jaimerz/Master/PLANCK/Planck_TTTEEE_lowl_lowE/base_plikHM_TTTEEE_lowl_lowE_post_zre6p5_3.txt')
planck4 = np.loadtxt('/home/jaimerz/Master/PLANCK/Planck_TTTEEE_lowl_lowE/base_plikHM_TTTEEE_lowl_lowE_post_zre6p5_4.txt')

planck_tot = np.concatenate((planck1, planck2, planck3, planck4), axis=0)

planck_params = []
for item in planck_tot:
    planck_params.append([item[29], item[32], item[34], item[35]])
planck_params = np.array(planck_params)
planck_samples = MCSamples(samples=planck_params, names = names_planck, labels = labels_planck, label= r'$Planck$ 2018',
                    weights=np.transpose(planck_tot)[0],loglikes=np.transpose(planck_tot)[1])

Removed no burn in


In [3]:
z_max = 1110
res = 200
x_arr = np.linspace(0, np.log(1+z_max), res)
dx = np.mean(np.diff(x_arr))
z_arr = np.exp(x_arr)-1
a_arr = 1./(1+z_arr)

path = '/home/jaimerz/PhD/Growz/data/products/'

data_class = MakeData(z_max, res , path)
Planck = data_class.Planck
z_planck = data_class.z_planck
c = data_class.c

DESI_data = data_class.get_DESI(new=True, mode=None)
WFIRST_data = data_class.get_WFIRST(new=False)
CC_data = data_class.get_CC(new=False)
DSS_data = data_class.get_DSS(new=False)
BOSS_data = data_class.get_BOSS(new=False)
eBOSS_data = data_class.get_eBOSS(new=False)
Wigglez_data = data_class.get_Wigglez(new=False)
DS17_data = data_class.get_DS17(new=False)
CMB_data = data_class.get_CMB(new=False)
FCMB_data = data_class.get_FCMB(new=False)

Using Planck mean


NameError: name 'classy' is not defined

In [ ]:
fig=plt.figure(figsize=(18,3.5))
fig.subplots_adjust(wspace=0.23)

ax1 = plt.subplot(131)
plt.plot(z_planck, Planck['Hkms_arr'], "k--", label='Planck')
plt.errorbar(CC_data['z'], CC_data['data'], yerr = CC_data['err'], fmt='bo', label='CC')
plt.errorbar(BOSS_data['z'], BOSS_data['para_data'], yerr=BOSS_data['para_err'], fmt='ro', label='BOSS DR12')
plt.errorbar(eBOSS_data['z'], c/1000/(eBOSS_data['para_data']*eBOSS_data['rd']), 
             yerr = 1/eBOSS_data['rd']/(np.array([0.47])), fmt='mo', label='eBOSS DR16')
# axis labels and title
plt.xlim(0, 2.5)
plt.ylim(50, 300)
plt.xlabel("z")
plt.ylabel(r"$H(z) \, \left(\frac{Kms/s}{Mpc} \right)$")
plt.legend();

ax1 = plt.subplot(132)
plt.plot(z_planck, Planck['dM_arr'], "k--", label='Planck')
plt.errorbar(DS17_data['z'], (10**((DS17_data['data']-25+19.4)/5))/(1+DS17_data['z']),
             yerr=(10**(DS17_data['err']/5))/(1+DS17_data['z']), fmt='co', label='DS17')
plt.errorbar(BOSS_data['z'], BOSS_data['perp_data'], yerr=BOSS_data['perp_err'], fmt='ro', label='BOSS DR12')
plt.errorbar(eBOSS_data['z'], eBOSS_data['perp_data']*eBOSS_data['rd'], 
             yerr=np.array([0.79])*eBOSS_data['rd'], fmt='mo', label='eBOSS DR16')
plt.errorbar(CMB_data['z'], (1/CMB_data['data'])*(100*CMB_data['rd']), 
             yerr=(1/CMB_data['err'])*1/(100*CMB_data['rd']), fmt='go', label='CMB')

plt.xlabel("z")
plt.ylabel(r"$D_M$")
plt.loglog()
plt.ylabel(r"$D_M \, (Mpc)$")
plt.legend();

ax1 = plt.subplot(133)
plt.plot(z_planck, Planck['f_arr']*Planck['s8_arr'], "k--", label='Planck')
plt.errorbar(BOSS_data['z'], BOSS_data['fs8_data'], yerr=BOSS_data['fs8_err'], fmt='ro', label='BOSS DR12')
plt.errorbar(eBOSS_data['z'], eBOSS_data['fs8_data'], yerr=eBOSS_data['fs8_err'], fmt='mo', label='eBOSS DR16')
plt.errorbar(Wigglez_data['z'], Wigglez_data['data'], yerr=Wigglez_data['err'], fmt='yo', label='Wigglez')
plt.errorbar(DSS_data['z'], DSS_data['data'], yerr=DSS_data['err'], fmt='go', label='DSS')

plt.xlim(-0.05, 2.5)
plt.ylim(0.3, 0.6)
plt.xlabel("z")
plt.ylabel(r"$f \sigma_8$")
plt.legend();

os.remove('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/data.pdf')
plt.savefig('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/data.pdf', bbox_inches='tight', pad_inches=0)


In [ ]:
def get_m_s(rows):
    cols = np.transpose(rows)
    means = np.array([])
    sigmas = np.array([])
    for col in cols:
        mean = np.mean(col)
        sigma = np.std(col)
        means = np.append(means, mean)
        sigmas = np.append(sigmas, sigma)
    return means, sigmas

In [ ]:
#LCDM 
path = '/home/jaimerz/PhD/Growz/data/outputs/LCDM/'
fiducial_LCDM = np.load(os.path.join(path,'LCDM_All_CMB_10000_10000/samples.npz'))
DESI_LCDM = np.load(os.path.join(path,'LCDM_DESI_CMB_10000_10000/samples.npz'))

path = '/home/jaimerz/PhD/Growz/data/outputs/finals/'
All_CMB = np.load(os.path.join(path, 'All_CMB_Planck_95_2500_2500/samples.npz'))
All_CMB_90 = np.load(os.path.join(path, 'All_CMB_Planck_90_2500_2500/samples.npz'))
All_CMB_hp = np.load(os.path.join(path,'All_CMB_Planck_l_0.001_n_0.1_3000_3000/samples.npz'))
All_CMB_noA0 = np.load(os.path.join(path, 'All_CMB_best_fit_noA0_3000_3000/samples.npz'))
All_CMB_NODSS = np.load(os.path.join(path, 'All_CMB_NODSS_Planck_5000_5000/samples.npz'))
All_CMB_bfWms8 = np.load(os.path.join(path, 'All_CMB_Planck_bfWms8_3000_3000/samples.npz'))
All_CMB_geo = np.load(os.path.join(path,'All_CMB_geo_Planck_5000_5000/samples.npz'))
All = np.load(os.path.join(path,'All_Planck_3000_3000/samples.npz'))
All_gro = np.load(os.path.join(path,'All_gro_Planck_A0_50000_50000/samples.npz'))
All_gro_wide = np.load(os.path.join(path,'All_gro_Planck_n_0.5_2500_2500/samples.npz'))
All_gro_wider = np.load(os.path.join(path,'All_gro_Planck_n_0.51_2500_2500/samples.npz'))
All_gro_widest = np.load(os.path.join(path,'All_gro_Planck_n_1_2500_2500/samples.npz'))
All_gro_diff_mean = np.load(os.path.join(path,'All_gro_Planckdiff_mean_n_0.5_2500_2500/samples.npz'))
DESI_CMB = np.load(os.path.join(path, 'DESI_CMB_Planck_3001_3001/samples.npz'))

# Numbers

In [ ]:
def table_numbers(samples, sample_names):
    no_cosmo = ['Geometry data', 'All data (BF $\Omega_m$ and $\sigma_8$)']
    no_dHz = ['All data ($\Lambda$CDM)', 'DESI $+$ CMB ($\Lambda$CDM)']
    table = {}
    for i, sample in enumerate(samples):
        print(sample_names[i])
        table[sample_names[i]] = {}
        
        if 'n' in sample.files  is not None:
            n = sample['n']
            n_m = np.mean(n)
            n_s = np.std(n)
        else:
            n_m = 0
            n_s = 0
        print('n', n_m, '+/-', n_s)
        table[sample_names[i]]['n_m'] = n_m
        table[sample_names[i]]['n_s'] = n_s
        
        if 'H0_gp' in sample.files:
            H0 = sample['H0_gp']
            H0_m = np.mean(H0)
            H0_s = np.std(H0)
        else:
            H0_m = 0
            H0_s = 0
        print('H0', H0_m, '+/-', H0_s)
        table[sample_names[i]]['H0_m'] = H0_m
        table[sample_names[i]]['H0_s'] = H0_s
        
        if 'S80' in sample.files and sample_names[i] not in no_cosmo:
            S80 = sample['S80']
            S80_m = np.mean(S80)
            S80_s = np.std(S80)
        else:
            S80_m = 0
            S80_s = 0
        print('S80', S80_m, '+/-', S80_s)
        table[sample_names[i]]['S80_m'] = S80_m
        table[sample_names[i]]['S80_s'] = S80_s
        
        if 's80' in sample.files and sample_names[i] not in no_cosmo:
            s80 = sample['s80']
            s80_m = np.mean(s80)
            s80_s = np.std(s80)
        else:
            s80_m = 0
            s80_s = 0
        print('s80', s80_m, '+/-', s80_s)
        table[sample_names[i]]['s80_m'] = s80_m
        table[sample_names[i]]['s80_s'] = s80_s
        
        if 'Omega_m' in sample.files and sample_names[i] not in no_cosmo:
            Wm = sample['Omega_m']
            Wm_m = np.mean(Wm)
            Wm_s = np.std(Wm)
        else:
            Wm_m = 0
            Wm_s = 0
        print('Wm0', Wm_m, '+/-', Wm_s)
        table[sample_names[i]]['Wm_m'] = Wm_m
        table[sample_names[i]]['Wm_s'] = Wm_s
        
        if 'DHz' in sample.files and sample_names[i] not in  no_dHz:
            varH = np.std(sample['DHz'],0)
            varH_m = np.mean(varH)
            varH_s = np.std(varH)
        else:
            varH = np.std(sample['Hz']/data_class.H_arr-1, 1)
            varH_m = np.mean(varH)
            varH_s = np.std(varH)
        print('varH', varH_m, '+/-', varH_s)
        table[sample_names[i]]['varH_m'] = varH_m
        table[sample_names[i]]['varH_s'] = varH_s
        print('-----')
        
    return table

In [ ]:
numbers = table_numbers([fiducial_LCDM, DESI_LCDM, All_CMB,  All_CMB_bfWms8, All_CMB_hp,   All_CMB_noA0,  All, 
               All_CMB_NODSS,   All_gro, All_CMB_geo, DESI_CMB],
              ['All data ($\Lambda$CDM)', 'DESI+CMB ($\Lambda$CDM)', 'All data', 'All data (BF $\Omega_m$ and $\sigma_8$)',
               'All data (Fixed HP)', 'All data (Two-Steps)', 'No CMB',
               'No DSS', 'Growth data', 'Geometry data', 'DESI $+$ CMB'])

In [ ]:
def make_table(numbers):
    names = numbers.keys()
    for name in names:
        Wm = '${} \pm {}$'.format(round(numbers[name]['Wm_m'],3), round(numbers[name]['Wm_s'],3))
        s80 = '${} \pm {}$'.format(round(numbers[name]['s80_m'],3), round(numbers[name]['s80_s'],3))
        S80 = '${} \pm {}$'.format(round(numbers[name]['S80_m'],3), round(numbers[name]['S80_s'],3))
        H0 = '${} \pm {}$'.format(round(numbers[name]['H0_m'],3), round(numbers[name]['H0_s'],3))
        print(name, '&', Wm, '&',  s80, '&',  S80, '&',  H0, r'\\' )

In [ ]:
def make_table2(numbers):
    names = numbers.keys()
    for name in names:
        varH = '${} \pm {}$'.format(round(numbers[name]['varH_m'],3), round(numbers[name]['varH_s'],3))
        n = '${} \pm {}$'.format(round(numbers[name]['n_m'],3), round(numbers[name]['n_s'],3))
        print(name, '&', n, '&', varH, r'\\' )

In [ ]:
make_table(numbers)

In [ ]:
make_table2(numbers)

# 1D Comp

In [ ]:
fig=plt.figure(figsize=(12,8))
fig.subplots_adjust(wspace=0)

my_yticks = ['All data ($\Lambda$CDM)', 'All data', 'All data (Fixed HP)', 'All data (Two-Steps)', 'No CMB',
               'No DSS', 'DESI $+$ CMB']

fmts = {'All data ($\Lambda$CDM)': 'k*',
        'All data': 'r*',
        'All data (Fixed HP)': 'g*',
        'All data (Two-Steps)': 'y*',
        'No CMB': 'm*',
        'No DSS': 'r*', 
        'DESI $+$ CMB': 'k*'}

y = {'All data ($\Lambda$CDM)': 0,
    'All data': 2,
    'All data (Fixed HP)': 4,
    'All data (Two-Steps)': 6,
    'No CMB': 8,
    'No DSS': 10, 
    'DESI $+$ CMB': 12}

ax1 = plt.subplot(131)
label = r'$\Omega_{\rm{m}}$'
plt.axvspan(0.3166 - 0.0084, 0.3166 + 0.0084, alpha = 0.5, color='b',  label= r'$Planck$ 2018')
for name in my_yticks:
    Wm=numbers[name]['Wm_m']
    Wm_err=numbers[name]['Wm_s']
    plt.errorbar( Wm, y[name], xerr=np.transpose(Wm_err), fmt=fmts[name])

plt.yticks(list(y.values()), list(y.keys()), fontsize=16)
plt.xlabel(label, fontsize=18)
plt.axis([0.1, 0.55, y[my_yticks[-1]]+1 ,-1,])
plt.yticks(rotation=15)
plt.xticks(fontsize=14)

    
ax2 = plt.subplot(132, sharey = ax1)
label = r'$\sigma_{\rm{8}}$'
plt.axvspan(0.8120 - 0.0073, 0.8120 + 0.0073, alpha = 0.5, color='b',  label= r'$Planck$ 2018')
for name in my_yticks:
    s8=numbers[name]['s80_m']
    s8_err=numbers[name]['s80_s']
    plt.errorbar( s8, y[name], xerr=np.transpose(s8_err), fmt=fmts[name])
plt.xlabel(label, fontsize=18)
plt.legend(loc = 1, prop={'size': 13})
plt.axis([0.6, 1.2, y[my_yticks[-1]]+1 ,-1,])
plt.setp( ax2.get_yticklabels(), visible=False)
plt.xticks(fontsize=14)
    
ax3 = plt.subplot(133, sharey = ax2)
label = r'$S_{\rm{8}}$'
plt.axvspan(0.834 - 0.016, 0.834 + 0.016, alpha = 0.5, color='b',  label= r'$Planck$ 2018')
for name in my_yticks:
    S8=numbers[name]['S80_m']
    S8_err=numbers[name]['S80_s']
    plt.errorbar( S8, y[name], xerr=np.transpose(S8_err), fmt=fmts[name])
plt.xlabel(label, fontsize=18)
#plt.legend(loc = 1, prop={'size': 13})
plt.axis([0.7, 0.9, y[my_yticks[-1]]+1 ,-1,])
plt.setp( ax3.get_yticklabels(), visible=False)
plt.xticks(fontsize=14)

os.remove('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/1D_comp.pdf')
plt.savefig('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/1D_comp.pdf', bbox_inches='tight', pad_inches=0)

# Triangle Plots

In [ ]:
def make_samples(file, names, ranges, label):
    namess = []
    labels = []
    samples = []
    for name in names:
        if name in file.keys():
            namess.append(name) 
            labels.append(labels_dict[name]) 
            samples.append(file[name]) 
        
    return MCSamples(samples=samples, names=namess, labels=labels, label=label, ranges=ranges,
                    settings={'mult_bias_correction_order':0,'smooth_scale_2D':0.4, 'smooth_scale_1D':0.3})

In [ ]:
labels_dict = {'n': '\eta',
               'l': 'l',
               'A0': 'A_0',
               'n_H': '\eta_H',
               'l_H': 'l_H',
               'n_Xi': '\eta_{Xi}',
               'l_Xi': 'l_{Xi}',
               'H0': 'H_0',
               'H0_gp': 'H_0',
               'omega_m': '\omega_m',
               'Omega_m': '\Omega_m',
               'Omega_m_mean': '\Omega_m^{mean}',
               'omega_b': '\omega_b',
               'Omega_m': '\Omega_m',
               'Omega_b': '\Omega_b',
               's80': '\sigma_8',
               'S80': 'S_8',
               'rd': 'r_s',
               'M': 'M'}
names_All = ['n', 'l', 'n_H', 'l_H', 'n_Xi', 'l_Xi', 'A0',
             'H0', 'H0_gp',  'Omega_m', 'omega_m', 'Omega_m_mean',
             'omega_b', 'Omega_m', 'Omega_b', 's80', 'S80', 'M']
ranges2 = {'l':[0.001, 7],}
ranges = {'l':[0.001, 7], 'Omega_m':[0.00, 0.5], 'n':[0, 1]}

In [ ]:
Fiducial_LCDM_samples = make_samples(fiducial_LCDM, ['H0_gp', 'Omega_m', 's80', 'S80', 'M', 'rd'], 
                                ranges, r'All data ($\Lambda$CDM)')
DESI_LCDM_samples = make_samples(DESI_LCDM, ['H0_gp', 'Omega_m', 's80', 'S80'], 
                                ranges, r'DESI+CMB ($\Lambda$CDM)')
All_CMB_samples = make_samples(All_CMB, ['A0', 'n', 'l', 'H0_gp', 'Omega_m', 's80', 'S80', 'rd', 'M'],
                               ranges, 'All data')
All_CMB_noA0_samples = make_samples(All_CMB_noA0, ['n', 'l', 'H0_gp', 'Omega_m', 's80', 'S80', 'rd', 'M'],
                               ranges, 'All data (Two-steps)')
All_CMB_hp_samples = make_samples(All_CMB_hp, ['A0', 'H0_gp', 'Omega_m', 's80', 'S80', 'rd', 'M'],
                               ranges, 'All data (Fixed HP)')
All_samples = make_samples(All, ['A0', 'n', 'l', 'H0_gp', 'Omega_m', 's80', 'S80', 'rd', 'M'],
                               ranges, 'No CMB')
All_CMB_NODSS_samples = make_samples(All_CMB_NODSS, ['A0', 'n', 'l', 'H0_gp', 'Omega_m', 's80', 'S80', 'rd', 'M'],
                               ranges, 'No DSS')
All_gro_samples = make_samples(All_gro, ['A0', 'n', 'l', 'H0_gp', 'Omega_m', 's80', 'S80'],
                               ranges, 'Growth data')
All_gro_wide_samples = make_samples(All_gro_wide, ['A0', 'l', 'H0_gp', 'Omega_m', 's80', 'S80'],
                               ranges, 'Growth data + $\eta=0.5$')
All_gro_wider_samples = make_samples(All_gro_wider, ['A0', 'l', 'H0_gp', 'Omega_m', 's80', 'S80'],
                               ranges, 'Growth data + $\eta=0.5$ + $A_0$ wide')
All_gro_widest_samples = make_samples(All_gro_widest, ['A0', 'l', 'H0_gp', 'Omega_m', 's80', 'S80'],
                               ranges, 'Growth data + $\eta=1$')
All_gro_diff_mean_samples = make_samples(All_gro_diff_mean, ['A0', 'l', 'H0_gp', 'Omega_m', 's80', 'S80'],
                               ranges, 'Growth data + diff mean')
All_CMB_geo_samples = make_samples(All_CMB_geo, ['A0', 'n', 'l', 'H0_gp'],
                               ranges, 'Geometry data')
All_CMB_bfWms8_samples = make_samples(All_CMB_bfWms8, ['n', 'l', 'H0_gp'],
                               ranges, 'All data (BF $\Omega_m$ and $\sigma_8$)')
DESI_CMB_samples = make_samples(DESI_CMB, ['A0','n', 'l', 'H0_gp', 'Omega_m', 's80', 'S80'], 
                                ranges, r' DESI+CMB')

In [ ]:
g = plots.getSubplotPlotter(subplot_size=2.5)
g.triangle_plot([All_gro_samples, All_gro_diff_mean_samples],
                ['A0', 'H0_gp', 'Omega_m', 's80', 'S80'],
                filled=True,
                markers={'H0':100*data_class.cosmo.h(),
                         'Omega_m': data_class.cosmo.Omega_m(),
                         's80': data_class.cosmo.sigma8(), 
                         'S80': data_class.cosmo.sigma8()*np.sqrt(data_class.cosmo.Omega_m()/0.3),
                         'rd':147.06546072,
                         'M': -19.2})             

In [ ]:
g = plots.getSubplotPlotter(subplot_size=2.5)
g.triangle_plot([All_CMB_samples, Fiducial_LCDM_samples],
                ['H0_gp', 'Omega_m', 's80', 'S80'],
                filled=True,
                markers={'H0':100*data_class.cosmo.h(),
                         'Omega_m': data_class.cosmo.Omega_m(),
                         's80': data_class.cosmo.sigma8(), 
                         'S80': data_class.cosmo.sigma8()*np.sqrt(data_class.cosmo.Omega_m()/0.3),
                         'rd':147.06546072,
                         'M': -19.2})                

In [ ]:
g = plots.getSubplotPlotter(subplot_size=2.5)
g.triangle_plot([Fiducial_LCDM_samples, DESI_CMB_samples, DESI_LCDM_samples],
                ['H0_gp', 'Omega_m', 's80', 'S80'],
                filled=True,
                markers={'H0':100*data_class.cosmo.h(),
                         'Omega_m': data_class.cosmo.Omega_m(),
                         's80': data_class.cosmo.sigma8(), 
                         'S80': data_class.cosmo.sigma8()*np.sqrt(data_class.cosmo.Omega_m()/0.3),
                         'rd':147.06546072,
                         'M': -19.2})
g.export('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/triangle_forecast.pdf')                

In [ ]:
g = plots.getSubplotPlotter(subplot_size=2.5)
g.triangle_plot([ All_gro_samples, All_CMB_samples],
                ['n'],
                filled=True,
                markers={'H0':100*data_class.cosmo.h(),
                         'Omega_m': data_class.cosmo.Omega_m(),
                         's80': data_class.cosmo.sigma8(), 
                         'S80': data_class.cosmo.sigma8()*np.sqrt(data_class.cosmo.Omega_m()/0.3),
                         'rd':147.06546072,
                         'M': -19.2})
                

In [ ]:
g = plots.getSubplotPlotter(subplot_size=2.5)
g.settings.axes_fontsize = 20
g.settings.legend_fontsize = 20
g.settings.axes_labelsize = 20
g.triangle_plot([All_CMB_noA0_samples, All_CMB_hp_samples, All_CMB_samples, Fiducial_LCDM_samples],
                filled=True,
                markers={'H0':100*data_class.cosmo.h(),
                         'Omega_m': data_class.cosmo.Omega_m(),
                         's80': data_class.cosmo.sigma8(), 
                         'S80': data_class.cosmo.sigma8()*np.sqrt(data_class.cosmo.Omega_m()/0.3),
                         'rd':147.06546072,
                         'M': -19.2})
g.export('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/triangle_tests.pdf')     

In [ ]:
g = plots.getSubplotPlotter(subplot_size=2.5)
g.settings.axes_fontsize = 20
g.settings.legend_fontsize = 20
g.settings.axes_labelsize = 20
g.triangle_plot([All_CMB_NODSS_samples, All_samples, All_CMB_samples, Fiducial_LCDM_samples],
                filled=True,
                markers={'H0':100*data_class.cosmo.h(),
                         'Omega_m': data_class.cosmo.Omega_m(),
                         's80': data_class.cosmo.sigma8(), 
                         'S80': data_class.cosmo.sigma8()*np.sqrt(data_class.cosmo.Omega_m()/0.3),
                         'rd':147.06546072,
                         'M': -19.2})
g.export('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/triangle_data.pdf')

## Challenge

In [ ]:
def challenge_1(cosmo):
    base_path = os.path.join('/home/jaimerz/PhD/Growz/data/outputs/finals/challenge/', cosmo)
    challenges = [f for f in listdir(base_path) if '.yml' not in f]#(not isfile(join(base_path, f))) or ('LCDM' not in f)]
    challenge_samples = []
    for challenge in challenges:
        path = os.path.join(base_path, challenge)+'/samples.npz'
        chain = np.load(path)
        samples = make_samples(chain, ['A0', 'n', 'l', 'H0_gp', 'Omega_m', 's80', 'S80', 'rd', 'M'], ranges2, challenge)
        challenge_samples.append(samples)
        with open(base_path+'/'+cosmo+'.yml') as f:
            ref = yaml.safe_load(f)
    g = plots.getSubplotPlotter(subplot_size=2.5)
    g.triangle_plot(challenge_samples,
                    ['H0_gp', 'Omega_m', 's80', 'S80'],
                filled=True,
                colors = ['red', 'grey', 'blue', 'green', 'magenta'],
                contour_colors = ['red', 'grey', 'blue', 'green', 'magenta'],
                markers={'H0_gp': ref['H0'],
                         'Omega_m': ref['Omega_m'],
                         's80': ref['sigma8'], 
                         'S80': ref['sigma8']*np.sqrt(ref['Omega_m']/0.3)})              
    g.export('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/triangle'+cosmo+'.pdf')
    return  

In [ ]:
challenge_1('cosmo1')

In [ ]:
challenge_1('cosmo2')

In [ ]:
challenge_1('cosmo3')

In [ ]:
challenge_1('cosmo4')

In [ ]:
challenge_1('cosmo5')

# d_H gp 

In [ ]:
dH_DESI_CMB_m, dH_DESI_CMB_s = get_m_s(DESI_CMB['DHz'])
dH_All_CMB_m, dH_All_CMB_s = get_m_s(All_CMB['DHz'])
dH_All_CMB_90_m, dH_All_CMB_90_s = get_m_s(All_CMB_90['DHz'])
dH_All_CMB_noA0_m, dH_All_CMB_noA0_s = get_m_s(All_CMB_noA0['DHz'])
dH_All_CMB_bfWms8_m, dH_All_CMB_bfWms8_s = get_m_s(All_CMB_bfWms8['DHz'])

dH_All_gro_m, dH_All_gro_s = get_m_s(All_gro['DHz'])
dH_All_CMB_geo_m, dH_All_CMB_geo_s = get_m_s(All_CMB_geo['DHz'])
dH_All_CMB_hp_m, dH_All_CMB_hp_s = get_m_s(All_CMB_hp['DHz'])

dH_All_m, dH_All_s = get_m_s(All['DHz'])
dH_All_CMB_NODSS_m, dH_All_CMB_NODSS_s = get_m_s(All_CMB_NODSS['DHz'])
dH_LCDM_m, dH_LCDM_s = get_m_s(1-fiducial_LCDM['Hz']/np.mean(fiducial_LCDM['Hz'], 0))

Planck = data_class.Planck
z_planck = data_class.z_planck

In [ ]:
fig=plt.figure(figsize=(8,12))
fig.subplots_adjust(hspace=0)
ax1 = plt.subplot(311)

plt.plot(z_planck, np.zeros_like(z_planck), 'k--')
plt.plot(z_arr,  dH_All_m, color ='b', label='No CMB')
plt.fill_between(z_arr, dH_All_m+dH_All_s,
                 dH_All_m-dH_All_s, color='b', alpha=.5)
plt.plot(z_arr,  dH_All_CMB_m, color ='r', label='All data')
plt.fill_between(z_arr, dH_All_CMB_m+dH_All_CMB_s,
                 dH_All_CMB_m-dH_All_CMB_s, color='r', alpha=.5)

plt.xscale('log')
plt.xlim(0, 1100)
plt.xlabel("z", fontsize=14 )
plt.ylabel(r"$ \delta H(z)$", fontsize=14)
plt.legend(loc='lower left')
plt.setp( ax1.get_xticklabels(), visible=False)
plt.yticks(fontsize=14)

ax2 = plt.subplot(312, sharex = ax1)
plt.plot(z_arr,  dH_All_CMB_m, color ='r', label='All data')
plt.fill_between(z_arr, dH_All_CMB_m+dH_All_CMB_s,
                 dH_All_CMB_m-dH_All_CMB_s, color='r', alpha=.5)

plt.plot(z_arr,  dH_All_gro_m, color ='m', label='Growth data')
plt.fill_between(z_arr, dH_All_gro_m+dH_All_gro_s,
                 dH_All_gro_m-dH_All_gro_s, color='m', alpha=.5)

plt.plot(z_arr,  dH_All_CMB_geo_m, color ='g', label='Geometry data')
plt.fill_between(z_arr, dH_All_CMB_geo_m+dH_All_CMB_geo_s,
                 dH_All_CMB_geo_m-dH_All_CMB_geo_s, color='g', alpha=.5)
plt.plot(z_planck, np.zeros_like(z_planck), 'k--')

plt.xscale('log')
plt.xlim(0, 1100)
#plt.ylim(-.04, .04)
plt.xlabel("z", fontsize=14 )
plt.ylabel(r"$ \delta H(z)$", fontsize=14)
#plt.title(r"$\delta H(z)$")
plt.legend(loc='lower left')
plt.setp( ax2.get_xticklabels(), visible=False)
plt.yticks(fontsize=14)

ax3 = plt.subplot(313, sharex = ax1)
#plt.plot(z_arr,  dH_All_CMB_m, color ='r', label='Fiducial')
#plt.fill_between(z_arr, dH_All_CMB_m+dH_All_CMB_s,
#                 dH_All_CMB_m-dH_All_CMB_s, color='r', alpha=.5)

plt.plot(z_arr,  dH_All_CMB_geo_m, color ='g', label='Geometry data')
plt.fill_between(z_arr, dH_All_CMB_geo_m+dH_All_CMB_geo_s,
                 dH_All_CMB_geo_m-dH_All_CMB_geo_s, color='g', alpha=.5)
plt.plot(z_arr,  dH_All_CMB_bfWms8_m, color ='k', label='All data (BF $\Omega_m$ $\sigma_8$)')
plt.fill_between(z_arr, dH_All_CMB_bfWms8_m+dH_All_CMB_bfWms8_s,
                 dH_All_CMB_bfWms8_m-dH_All_CMB_bfWms8_s, color='k', alpha=.5)
#plt.plot(z_arr,  dH_All_CMB_m, color ='r', label='Fiducial')
#plt.fill_between(z_arr, dH_All_CMB_m+dH_All_CMB_s,
#                 dH_All_CMB_m-dH_All_CMB_s, color='r', alpha=.5)
#plt.plot(z_arr,  dH_All_CMB_NODSS_m, color ='r', label='Fiducial')
#plt.fill_between(z_arr, dH_All_CMB_NODSS_m+dH_All_CMB_NODSS_s,
#                 dH_All_CMB_NODSS_m-dH_All_CMB_NODSS_s, color='r', alpha=.5)

plt.plot(z_planck, np.zeros_like(z_planck), 'k--')

plt.xscale('log')
plt.xlim(0, 1100)
#plt.ylim(-.04, .04)
plt.xlabel("z", fontsize=14 )
plt.ylabel(r"$ \delta H(z)$", fontsize=14)
plt.legend(loc='lower left')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)



os.remove('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/dH_gp_geo_vs_gro.pdf')
plt.savefig('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/dH_gp_geo_vs_gro.pdf', box_inches='tight', pad_inches=0)  

In [ ]:
plt.plot(z_arr,  dH_All_CMB_m, color ='r', label='All data')
plt.fill_between(z_arr, dH_All_CMB_m+dH_All_CMB_s,
                 dH_All_CMB_m-dH_All_CMB_s, color='r', alpha=.5)

plt.plot(z_arr,  dH_All_CMB_hp_m, color ='g', label='All data (Fixed HP)')
plt.fill_between(z_arr, dH_All_CMB_hp_m+dH_All_CMB_hp_s,
                 dH_All_CMB_hp_m-dH_All_CMB_hp_s, color='g', alpha=.5)

#plt.plot(z_arr,  dH_All_CMB_hp_m, color ='k', label='Fixed HP')
#plt.fill_between(z_arr, dH_All_CMB_hp_m+dH_All_CMB_hp_s,
#                 dH_All_CMB_hp_m-dH_All_CMB_hp_s, color='k', alpha=.5)

plt.plot(z_arr,  dH_All_CMB_noA0_m, color ='b', label='All data (Two-Steps)')
plt.fill_between(z_arr, dH_All_CMB_noA0_m+dH_All_CMB_noA0_s,
                 dH_All_CMB_noA0_m-dH_All_CMB_noA0_s, color='b', alpha=.5)

plt.plot(z_planck, np.zeros_like(z_planck), 'k--')


plt.xscale('log')
plt.xlim(0, 1100)
plt.xlabel("z")
plt.ylabel(r"$ \delta H(z)$")
plt.legend();

os.remove('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/dH_gp_tests.pdf')
plt.savefig('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/dH_gp_tests.pdf', bbox_inches='tight',  pad_inches=0)  

In [ ]:
#fig=plt.figure(figsize=(6*(7/8),8*(7/8)))
#fig.subplots_adjust(hspace=0)
#ax1 = plt.subplot(211)

plt.plot(z_arr,  dH_All_CMB_m, color ='r', label='All data')
plt.fill_between(z_arr, dH_All_CMB_m+dH_All_CMB_s,
                 dH_All_CMB_m-dH_All_CMB_s, color='r', alpha=.5)

plt.plot(z_arr,  dH_DESI_CMB_m, color ='b', label='DESI+CMB')
plt.fill_between(z_arr, dH_DESI_CMB_m+dH_DESI_CMB_s,
                 dH_DESI_CMB_m-dH_DESI_CMB_s, color='b', alpha=.5)
plt.plot(z_planck, np.zeros_like(z_planck), 'k--')


plt.xscale('log')
plt.xlim(0, 1100)
plt.xlabel("z")
plt.ylabel(r"$ \delta H(z)$")
plt.setp( ax1.get_xticklabels(), visible=False)
plt.legend();

#ax2=plt.subplot(212)

#plt.plot(z_arr, np.std(np.transpose(DESI_CMB['Hz']/data_class.H_arr)/DESI_CMB['A0']-1, 1), 'b', label='GP - DESI+CMB')
#plt.fill_between(z_arr,np.std(np.transpose(DESI_CMB['Hz']/data_class.H_arr)/DESI_CMB['A0']-1, 1), 0, color='b', alpha=0.5)

#plt.plot(z_arr, np.std(np.transpose(fiducial_LCDM['Hz']/np.mean(fiducial_LCDM['Hz'], 0))-1, 1), 'g', label='$\Lambda$CDM - Current data')
#plt.fill_between(z_arr, np.std(np.transpose(fiducial_LCDM['Hz']/np.mean(fiducial_LCDM['Hz'], 0))-1, 1), 0, color='g', alpha=0.5)

#plt.xscale('log')
#plt.ylabel(r"$ \delta H(z)$")
#plt.xlabel('z')
#plt.legend()
#plt.xlim(0, 1100)
#plt.ylim(0, 0.037);

os.remove('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/dH_gp_forecast.pdf')
plt.savefig('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/dH_gp_forecast.pdf', bbox_inches='tight',  pad_inches=0)  

In [ ]:
plt.plot(z_arr,  dH_All_CMB_m, color ='r', label='All data (TAP = $95\%$)')
plt.fill_between(z_arr, dH_All_CMB_m+dH_All_CMB_s,
                 dH_All_CMB_m-dH_All_CMB_s, color='r', alpha=.5)

plt.plot(z_arr,  dH_All_CMB_90_m, color ='b', label='All data (TAP = $90\%$)')
plt.fill_between(z_arr, dH_All_CMB_90_m+dH_All_CMB_90_s,
                 dH_All_CMB_90_m-dH_All_CMB_90_s, color='b', alpha=.5)

plt.plot(z_planck, np.zeros_like(z_planck), 'k--')

plt.xscale('log')
plt.xlim(0, 1100)
plt.xlabel("z")
plt.ylabel(r"$ \delta H(z)$")
plt.legend();

os.remove('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/90_vs_95.pdf')
plt.savefig('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/90_vs_95.pdf', bbox_inches='tight',  pad_inches=0)  

# H_gp, DM_gp and f$\sigma_8$_gp

## Load samples

In [ ]:
H_DESI_CMB_m, H_DESI_CMB_s = get_m_s(DESI_CMB['Hz'])
H_All_CMB_m, H_All_CMB_s = get_m_s(All_CMB['Hz'])
H_All_CMB_noA0_m, H_All_CMB_noA0_s = get_m_s(All_CMB_noA0['Hz'])
H_All_CMB_hp_m, H_All_CMB_hp_s = get_m_s(All_CMB_hp['Hz'])
H_All_gro_m, H_All_gro_s = get_m_s(All_gro['Hz'])
H_All_m, H_All_s = get_m_s(All['Hz'])
H_All_CMB_NODSS_m, H_All_CMB_NODSS_s = get_m_s(All_CMB_NODSS['Hz'])

In [ ]:
dM_DESI_CMB_m, dM_DESI_CMB_s = get_m_s(DESI_CMB['dMz'])
dM_All_CMB_m, dM_All_CMB_s = get_m_s(All_CMB['dMz'])
dM_All_CMB_noA0_m, dM_All_CMB_noA0_s = get_m_s(All_CMB_noA0['dMz'])
dM_All_CMB_hp_m, dM_All_CMB_hp_s = get_m_s(All_CMB_hp['dMz'])
dM_All_gro_m, dM_All_gro_s = get_m_s(All_gro['dMz'])
dM_All_m, dM_All_s = get_m_s(All['dMz'])
dM_All_CMB_NODSS_m, dM_All_CMB_NODSS_s = get_m_s(All_CMB_NODSS['dMz'])

In [ ]:
fs8_DESI_CMB_m, fs8_DESI_CMB_s = get_m_s(DESI_CMB['fs8z'])
fs8_All_CMB_m, fs8_All_CMB_s = get_m_s(All_CMB['fs8z'])
fs8_All_CMB_noA0_m, fs8_All_CMB_noA0_s = get_m_s(All_CMB_noA0['fs8z'])
fs8_All_CMB_hp_m, fs8_All_CMB_hp_s = get_m_s(All_CMB_hp['fs8z'])
fs8_All_gro_m, fs8_All_gro_s = get_m_s(All_gro['fs8z'])
fs8_All_m, fs8_All_s = get_m_s(All['fs8z'])
fs8_All_CMB_NODSS_m, fs8_All_CMB_NODSS_s = get_m_s(All_CMB_NODSS['fs8z'])
fs8_All_CMB_bfWms8_m, fs8_All_CMB_bfWms8_s = get_m_s(All_CMB_bfWms8['fs8z'])

## Plots

In [ ]:
fig=plt.figure(figsize=(7,7))
fig.subplots_adjust(hspace=0)

ax1 = plt.subplot(211)
plt.plot(z_planck, Planck['Hkms_arr'], 'k--', label='Planck')
plt.errorbar(CC_data['z'], CC_data['data'], yerr = CC_data['err'], fmt='bo')#, label='CC')
plt.errorbar(BOSS_data['z'], BOSS_data['para_data'], yerr=BOSS_data['para_err'], fmt='bs')#, label='BOSS DR12')
plt.errorbar(eBOSS_data['z'], c/1000/(eBOSS_data['para_data']*eBOSS_data['rd']), 
             yerr = 1/eBOSS_data['rd']/(np.array([0.47])), fmt='ks')#, label='eBOSS DR16')

plt.plot(z_arr,  H_All_CMB_m, color ='r', label='All data')
plt.fill_between(z_arr, H_All_CMB_m+H_All_CMB_s, H_All_CMB_m-H_All_CMB_s, color='r', alpha=.5)
plt.plot(z_arr,  H_All_CMB_hp_m, color ='k', label='All data (Fixed HP)')
plt.fill_between(z_arr, H_All_CMB_hp_m+H_All_CMB_hp_s,
                 H_All_CMB_hp_m-H_All_CMB_hp_s, color='k', alpha=.5)
plt.plot(z_arr,  H_All_CMB_noA0_m, color ='b', label='All data (Two-Steps)')
plt.fill_between(z_arr, H_All_CMB_noA0_m+H_All_CMB_noA0_s,
                 H_All_CMB_noA0_m-H_All_CMB_noA0_s, color='b', alpha=.5)
plt.xlim(0, 2.5)
plt.ylim(50, 300)
plt.xlabel("z")
plt.ylabel(r"$H(z) \, \left(\frac{Kms/s}{Mpc} \right)$")
plt.xticks(visible=False)
plt.legend();

#ax2 = plt.subplot(132)
#plt.plot(z_planck, Planck['dM_arr'], 'k--', label='Planck')
#plt.errorbar(DS17_data['z'], (10**((DS17_data['data']-25+19.49)/5))/(1+DS17_data['z']),
#             yerr=(10**(DS17_data['err']/5))/(1+DS17_data['z']), fmt='ko', label='DS17')
#plt.errorbar(BOSS_data['z'], BOSS_data['perp_data'], yerr=BOSS_data['perp_err'], fmt='ks', label='BOSS DR12')
#plt.errorbar(eBOSS_data['z'], eBOSS_data['perp_data']*eBOSS_data['rd'],
#             yerr=np.array([0.79])*eBOSS_data['rd'], fmt='ks', label='eBOSS DR16')
#plt.errorbar(CMB_data['z'], (1/CMB_data['data'])*(100*CMB_data['rd']),
#
#plt.plot(z_arr,  dM_All_CMB_m, color ='r', label='All data')
#plt.fill_between(z_arr, dM_All_CMB_m+dM_All_CMB_s, dM_All_CMB_m-dM_All_CMB_s, color='r', alpha=.5)
#plt.plot(z_arr,  dM_All_CMB_hp_m, color ='k', label='Fixed HP')
#plt.fill_between(z_arr, dM_All_CMB_hp_m+dM_All_CMB_hp_s,
#                 dM_All_CMB_hp_m-dM_All_CMB_hp_s, color='k', alpha=.5)
#plt.plot(z_arr,  dM_All_CMB_noA0_m, color ='b', label='Hierarchical')
#plt.fill_between(z_arr, dM_All_CMB_noA0_m+dM_All_CMB_noA0_s,
#                dM_All_CMB_noA0_m-dM_All_CMB_noA0_s, color='b', alpha=.5)

#plt.xlabel("z")
#plt.xscale('log')
#plt.yscale('log')
#plt.ylabel(r"$D_M$")
#plt.title(r"$D_M \, (Mpc)$")
#plt.legend();

ax3 = plt.subplot(212)
plt.plot(z_planck, Planck['fs8_arr'], 'k--', label='Planck')
plt.errorbar(BOSS_data['z'], BOSS_data['fs8_data'], yerr=BOSS_data['fs8_err'], fmt='bs')#, label='BOSS DR12')
plt.errorbar(eBOSS_data['z'], eBOSS_data['fs8_data'], yerr=eBOSS_data['fs8_err'], fmt='bs')#, label='eBOSS DR16')
plt.errorbar(Wigglez_data['z'], Wigglez_data['data'], yerr=Wigglez_data['err'], fmt='bo')#, label='Wigglez')
plt.errorbar(DSS_data['z'], DSS_data['data'], yerr=DSS_data['err'], fmt='ko')#, label='DSS')


plt.plot(z_arr,  fs8_All_CMB_noA0_m, color ='b', label='All data (Two-Steps)')
plt.fill_between(z_arr, fs8_All_CMB_noA0_m+fs8_All_CMB_noA0_s,
                 fs8_All_CMB_noA0_m-fs8_All_CMB_noA0_s, color='b', alpha=.5)
plt.plot(z_arr,  fs8_All_CMB_hp_m, color ='k', label='All data (Fixed HP)')
plt.fill_between(z_arr, fs8_All_CMB_hp_m+fs8_All_CMB_hp_s,
                 fs8_All_CMB_hp_m-fs8_All_CMB_hp_s, color='k', alpha=.5)
plt.plot(z_arr,  fs8_All_CMB_m, color ='r', label='All data')
plt.fill_between(z_arr, fs8_All_CMB_m+fs8_All_CMB_s,
                 fs8_All_CMB_m-fs8_All_CMB_s, color='r', alpha=.5)


plt.xlim(-0.01, 2.5)
plt.ylim(0.25, 0.55)
plt.xlabel("z")
plt.ylabel(r"$f \sigma_8$")
plt.legend();



os.remove('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/tests_cosmo_functions.pdf')
plt.savefig('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/tests_cosmo_functions.pdf', bbox_inches='tight',  pad_inches=0)   

In [ ]:
fig=plt.figure(figsize=(7,7))
fig.subplots_adjust(hspace=0)

ax1 = plt.subplot(211)
plt.plot(z_planck, Planck['Hkms_arr'], 'k--', label='Planck')

plt.errorbar(CC_data['z'], CC_data['data'], yerr = CC_data['err'], fmt='bo')#, label='CC')
plt.errorbar(BOSS_data['z'], BOSS_data['para_data'], yerr=BOSS_data['para_err'], fmt='bs')#, label='BOSS DR12')
plt.errorbar(eBOSS_data['z'], c/1000/(eBOSS_data['para_data']*eBOSS_data['rd']), 
             yerr = 1/eBOSS_data['rd']/(np.array([0.47])), fmt='bs')#, label='eBOSS DR16')
plt.plot(z_arr,  H_All_CMB_m, color ='r', label='All data')
plt.fill_between(z_arr, H_All_CMB_m+H_All_CMB_s,
                 H_All_CMB_m-H_All_CMB_s, color='r', alpha=.5)
plt.plot(z_arr,  H_All_m, color ='g', label='No CMB')
plt.fill_between(z_arr, H_All_m+H_All_s, 
                 H_All_m-H_All_s, color='g', alpha=.5)
plt.plot(z_arr,  H_All_CMB_NODSS_m, color ='k', label='No DSS')
plt.fill_between(z_arr, H_All_CMB_NODSS_m+H_All_CMB_NODSS_s,
                 H_All_CMB_NODSS_m-H_All_CMB_NODSS_s, color='k', alpha=.5)
plt.xlim(0, 2.5)
plt.ylim(50, 300)
plt.xlabel("z")
plt.ylabel(r"$H(z) \, \left(\frac{Kms/s}{Mpc} \right)$")
plt.xticks(visible=False)
plt.legend();

#ax2 = plt.subplot(132)
#plt.plot(z_planck, Planck['dM_arr'], 'k--', label='Planck')
#plt.errorbar(DS17_data['z'], (10**((DS17_data['data']-25+19.49)/5))/(1+DS17_data['z']),
#             yerr=(10**(DS17_data['err']/5))/(1+DS17_data['z']), fmt='bo', label='DS17')
#plt.errorbar(BOSS_data['z'], BOSS_data['perp_data'], yerr=BOSS_data['perp_err'], fmt='bs', label='BOSS DR12')
#plt.errorbar(eBOSS_data['z'], eBOSS_data['perp_data']*eBOSS_data['rd'],
#             yerr=np.array([0.79])*eBOSS_data['rd'], fmt='bs', label='eBOSS DR16')
#plt.errorbar(CMB_data['z'], (1/CMB_data['data'])*(100*CMB_data['rd']),
#             yerr=(1/CMB_data['err'])*1/(100*CMB_data['rd']), fmt='bo', label='CMB')
#plt.plot(z_arr,  dM_All_CMB_m, color ='r', label='All data')
#plt.fill_between(z_arr, dM_All_CMB_m+dM_All_CMB_s,
#                 dM_All_CMB_m-dM_All_CMB_s, color='r', alpha=.5)
#plt.plot(z_arr,  dM_All_m, color ='g', label='No CMB')
#plt.fill_between(z_arr, dM_All_m+dM_All_s, 
#                 dM_All_m-dM_All_s, color='g', alpha=.5)
#plt.plot(z_arr,  dM_All_CMB_NODSS_m, color ='k', label='No DSS')
#plt.fill_between(z_arr, dM_All_CMB_NODSS_m+dM_All_CMB_NODSS_s,
#                 dM_All_CMB_NODSS_m-dM_All_CMB_NODSS_s, color='k', alpha=.5)
#plt.xlabel("z")
#plt.ylabel(r"$D_M$")
#plt.xscale('log')
#plt.yscale('log')
#plt.title(r"$D_M \, (Mpc)$")
#plt.legend();

ax2 = plt.subplot(212)
plt.plot(z_planck, Planck['fs8_arr'], 'k--', label='Planck')
plt.errorbar(BOSS_data['z'], BOSS_data['fs8_data'], yerr=BOSS_data['fs8_err'], fmt='bs')#, label='BOSS DR12')
plt.errorbar(eBOSS_data['z'], eBOSS_data['fs8_data'], yerr=eBOSS_data['fs8_err'], fmt='bs')#, label='eBOSS DR16')
plt.errorbar(Wigglez_data['z'], Wigglez_data['data'], yerr=Wigglez_data['err'], fmt='bo')#, label='Wigglez')
plt.errorbar(DSS_data['z'], DSS_data['data'], yerr=DSS_data['err'], fmt='bo')#, label='DSS')
plt.plot(z_arr,  fs8_All_CMB_m, color ='r', label='All data')
plt.fill_between(z_arr, fs8_All_CMB_m+fs8_All_CMB_s,
                 fs8_All_CMB_m-fs8_All_CMB_s, color='r', alpha=.5)
plt.plot(z_arr,  fs8_All_m, color ='g', label='No CMB')
plt.fill_between(z_arr, fs8_All_m+fs8_All_s, 
                 fs8_All_m-fs8_All_s, color='g', alpha=.5)
plt.plot(z_arr,  fs8_All_CMB_NODSS_m, color ='k', label='No DSS')
plt.fill_between(z_arr, fs8_All_CMB_NODSS_m+fs8_All_CMB_NODSS_s,
                 fs8_All_CMB_NODSS_m-fs8_All_CMB_NODSS_s, color='k', alpha=.5)

plt.xlim(-0.001, 2.5)
plt.ylim(0.25, 0.55)
plt.xlabel("z")
plt.ylabel(r"$f \sigma_8$")
plt.legend();

os.remove('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/data_cosmo_functions.pdf')
plt.savefig('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/data_cosmo_functions.pdf', bbox_inches='tight',  pad_inches=0)  

In [ ]:
plt.plot(z_planck, Planck['fs8_arr'], 'k--', label='Planck')
plt.errorbar(BOSS_data['z'], BOSS_data['fs8_data'], yerr=BOSS_data['fs8_err'], fmt='ks', label='BOSS DR12')
plt.errorbar(eBOSS_data['z'], eBOSS_data['fs8_data'], yerr=eBOSS_data['fs8_err'], fmt='ks', label='eBOSS DR16')
plt.errorbar(Wigglez_data['z'], Wigglez_data['data'], yerr=Wigglez_data['err'], fmt='ko', label='Wigglez')
plt.errorbar(DSS_data['z'], DSS_data['data'], yerr=DSS_data['err'], fmt='ko', label='DSS')

plt.plot(z_arr,  fs8_All_CMB_m, color ='b', label='All data')
plt.fill_between(z_arr, fs8_All_CMB_m+fs8_All_gro_s,
                 fs8_All_CMB_m-fs8_All_CMB_s, color='b', alpha=.5)

plt.plot(z_arr,  fs8_All_CMB_bfWms8_m, color ='m', label='All data + BF $\Omega_m$ $\sigma_8$')
plt.fill_between(z_arr, fs8_All_CMB_bfWms8_m+fs8_All_CMB_bfWms8_s,
                 fs8_All_CMB_bfWms8_m-fs8_All_CMB_bfWms8_s, color='m', alpha=.5)


plt.xlim(-0.05, 2.5)
plt.ylim(0.25, 0.55)
plt.xlabel("z")
plt.ylabel(r"$f \sigma_8$")
plt.title(r"$f \sigma_8$")
plt.legend();

os.remove('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/fs8_Wms8_effect.pdf')
plt.savefig('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/fs8_Wms8_effect.pdf', bbox_inches='tight',  pad_inches=0)  

In [ ]:
plt.plot(z_planck, Planck['fs8_arr'], 'k--', label='Planck')
plt.errorbar(BOSS_data['z'], BOSS_data['fs8_data'], yerr=BOSS_data['fs8_err'], fmt='ks', label='BOSS DR12')
plt.errorbar(eBOSS_data['z'], eBOSS_data['fs8_data'], yerr=eBOSS_data['fs8_err'], fmt='ks', label='eBOSS DR16')
plt.errorbar(Wigglez_data['z'], Wigglez_data['data'], yerr=Wigglez_data['err'], fmt='ko', label='Wigglez')
plt.errorbar(DSS_data['z'], DSS_data['data'], yerr=DSS_data['err'], fmt='ko', label='DSS')

plt.plot(z_arr,  fs8_All_CMB_m, color ='r', label='All data')
plt.fill_between(z_arr, fs8_All_CMB_m+fs8_All_gro_s,
                 fs8_All_CMB_m-fs8_All_CMB_s, color='r', alpha=.5)

plt.plot(z_arr,  fs8_All_gro_m, color ='b', label='Growth data')
plt.fill_between(z_arr, fs8_All_gro_m+fs8_All_gro_s,
                 fs8_All_gro_m-fs8_All_gro_s, color='b', alpha=.5)


plt.xlim(-0.05, 2.5)
plt.ylim(0.25, 0.55)
plt.xlabel("z")
plt.ylabel(r"$f \sigma_8$")
plt.title(r"$f \sigma_8$")
plt.legend();

os.remove('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/fs8_wm_effect.pdf')
plt.savefig('/mnt/c/Users/Jaime/Desktop/PhD/gp_figures/fs8_wm_effect.pdf', bbox_inches='tight',  pad_inches=0)  